In [ ]:
import urllib.request
import urllib.parse
import tempfile
import scrapbook
import uuid
import xarray as xr

In [ ]:
query_string = urllib.parse.urlencode(args)
url = f"https://edc-oapi.hub.eox.at/oapi/collections/{data}/coverage?{query_string}"

print(f"Retrieving data from {url}")

with urllib.request.urlopen(url) as in_file:
    with tempfile.NamedTemporaryFile(suffix=".tif", delete=False) as tmp_file:
        tmp_file.write(in_file.read())
        tmp_file.flush()
        arr = xr.open_rasterio(tmp_file.name)


In [ ]:
ds = arr.to_dataset("band").rename({1: "B04", 2: "B08"})
print(f"Executing code: {code}")
exec(code)
print("got result:")
print(result)

In [ ]:
!python3 -m pip -q install rioxarray
import rioxarray

output_filepath = f"/home/jovyan/s3/ndvi-{uuid.uuid4()}.tif"

print(f"Writing output to {output_filepath}")

result.rio.to_raster(output_filepath)

scrapbook.glue("result-file", output_filepath)